# Things to Try Out or Learn
* GridSearch, parameter optimization.
* Implement Stacking technique
* LabelBinarizer()
* LabelEncoder()
* LogisticRegression()
* Solve 'attributerror: lower not found' problem.
* What is LogLoss?
* What is astype('U')?
* What is model.predict_proba?
* Keras Documentation

* Do something about the dates in the comments.

In [43]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.metrics import mean_squared_log_error
import string
from nltk.corpus import stopwords
%matplotlib inline
os.chdir("C:/Users/dogus/Dropbox/DgsPy_DBOX/Hepsiburada/")

# Age groups
# Gender Groups

def load_pickle(fname):
    with open('cins/' + fname + '.pk', 'rb') as f:
        return pickle.load(f)
    
from contextlib import contextmanager
import time

@contextmanager
def timer(thing):
    t0 = time.time()
    yield
    print(f'[{thing}] done in {time.time()-t0:.0f} s')

In [2]:
data = pd.read_csv('hepsib_cellphone_comments4.csv')
df = data.copy()


In [3]:
df.head()

,Date,Name,Age,Location,CommentTitle,Comment,Rating,p_helpful,n_helpful,link,Brand,Gender
0,12.03.2018,TOLGA ALKAN,28.0,NaN,Güzel Telefon,23.08.2016 da siparişini verdim. 26.08.2016 da...,80,2,5,/sony-xperia-xa-ultra-sony-turkiye-garantili-p...,sony,M
1,15.02.2018,Mehmet Karakoyun,NaN,-,Beş Yıldız,Harika bir telefon 2017yılbaşından birkaçgün ö...,100,3,4,/sony-xperia-xa-ultra-sony-turkiye-garantili-p...,sony,M
2,08.02.2018,Ümit Ertan,28.0,- Ankara - Türkiye,çok kötü,Şimdiye kadar satın aldığım en kalitesiz telef...,20,6,15,/sony-xperia-xa-ultra-sony-turkiye-garantili-p...,sony,U
3,30.01.2018,ismail ÖZKAN,NaN,-,orta,Telefon Guzel 2 eksigi var batarya az gidiyor ...,60,1,3,/sony-xperia-xa-ultra-sony-turkiye-garantili-p...,sony,NaN
4,23.01.2018,Nihal Erkal,NaN,NaN,Biraz iyi baya kötü,Kamerası güzel fiyatı uygun diye aldım ancak t...,40,10,3,/sony-xperia-xa-ultra-sony-turkiye-garantili-p...,sony,F


In [4]:
df['Comment'][3].lower()

'telefon guzel 2 eksigi var batarya az gidiyor ve 16gb başka kusuru yok hattiniz orjinal degil sonradan kesme ise tam oturmadigindan sınyal sikintisi oluyor dikkat'

In [7]:
# Create a Model column
df['Model'] = df['link'].apply(lambda x: x.split('-')[1]+"-"+x.split('-')[2])

In [8]:
# Adjust the 'Location' column
def location_adjustment(x):
    if isinstance(x, float):
        pass
    else:
        x = x.replace('-',' ').replace('Türkiye','')
        return x
df['Location'] = df['Location'].apply(location_adjustment)

In [9]:
def gender_sort(x):
    if isinstance(x, float) or x=='U':
        return 'Unknown'
    elif x=='M':
        return 'Male'
    elif x=='F':
        return 'Female'
    else:
        print('Something went wrong in gender_sort:',x)
        return x
df['Gender'] = df['Gender'].apply(gender_sort)

In [10]:
def text_clean(text):
    # Clean out all the stopwords
    stext = [word for word in text.split() if word.lower() not in stopwords.words('turkish')]
    ftext = ' '.join(stext)
    
    # Clean out all the punctuations
    stext = [word for word in ftext if word not in string.punctuation]
    ftext = ''.join(stext)
    return ftext
t0 = time.time()
df['Comment'] = df['Comment'].apply(text_clean)
print("Completed within %0.1f seconds." % (time.time() - t0))

Completed within 263.4 seconds.


In [24]:
def date6_delete(x):
    # Clean all the 8 digit values.
    comment = list()
    for line in x.split():
        if (len(line)==8) and ('20' in line):
            pass
        else:
            comment.append(line)
    return " ".join(comment)

df['Comment'] = df['Comment'].apply(date6_delete)

In [ ]:
"""def date_delete(x):
    """

In [64]:
# Delete everything except for alphabet and numbers ??
alphabet = "qwertyuıopğüasdfghjklşizxcvbnmöç"
numbers = "1234567890"

'1234567890'

In [25]:
# Drop the unnecessary columns: Date, Name, link, (helpfuls, location perhaps?)
raw = df.drop(['Date', 'Name', 'link','p_helpful', 'n_helpful'],axis=1)
raw.head()

,Age,Location,CommentTitle,Comment,Rating,Brand,Gender,Model
0,28.0,None,Güzel Telefon,siparişini verdim telefon elime ulaştıOrjinal ...,80,sony,Male,xperia-xa
1,NaN,,Beş Yıldız,Harika bir telefon 2017yılbaşından birkaçgün ö...,100,sony,Male,xperia-xa
2,28.0,Ankara,çok kötü,Şimdiye kadar satın aldığım kalitesiz telefon ...,20,sony,Unknown,xperia-xa
3,NaN,,orta,Telefon Guzel 2 eksigi var batarya gidiyor 16g...,60,sony,Unknown,xperia-xa
4,NaN,None,Biraz iyi baya kötü,Kamerası güzel fiyatı uygun aldım ancak tavsiy...,40,sony,Female,xperia-xa


In [ ]:
"""# Discard brands with less than 20 comments
a = df[(df['Model'].value_counts()>20)]

df2 = df.loc[:,'Model'].apply(lambda x: True)
#df2.loc[:,'Model'].value_counts()"""

In [27]:
# Create a overall text ready for CountVectorizer, Tfidf
raw['Text'] = raw['Location'].fillna('')+' '+raw['CommentTitle'].fillna('')+' '+raw['Comment'].fillna('') \
        +' '+raw['Brand'].fillna('')+' '+raw['Model'].fillna('')+' '+raw['Gender'].fillna('')

In [28]:
raw['Text']

0         Güzel Telefon siparişini verdim telefon elime...
1            Beş Yıldız Harika bir telefon 2017yılbaşın...
2           Ankara    çok kötü Şimdiye kadar satın aldı...
3            orta Telefon Guzel 2 eksigi var batarya gi...
4         Biraz iyi baya kötü  Kamerası güzel fiyatı uy...
5         KRONİK SIKINTI VAR TELEFON KULLANIM AÇISINDAN...
6        BEŞİKTAŞ   İstanbul    Bir Yıldız tek kelime b...
7           İçel    EKRAN SIKINTILI  TELEFON ON NUMARA ...
8           İstanbul    Beş Yıldız İstanbula 1 günde ka...
9         TELEFON ÇOK GÜZEL ŞEKİLDE TAVSİYE ETMEM BEDEN...
10          İstanbul    Fena değil  Ürün güzel ancak çi...
11          Van    batarya 6 İNÇ FULL HD BİR TELEFON İÇ...
12        Hızlı kargo , cok iyi kamera Hızlı geldi tele...
13        Xa ultra ekran problemi standart  Sony telefo...
14       ÇANKAYA   Ankara    Almadan iyi dusunun Cihazi...
15        Üç Yıldız 60 inç telefona okadar batarya koyu...
16       MERKEZ   Sivas    cok kotu asiri isinma cekmiy.

# Plots

In [ ]:
#sns.distplot(df['Rating'], bins=100) # categorical target, is standardscaler suitable? 

In [201]:
type(bag_of_words(shape=())

scipy.sparse.csr.csr_matrix

# Vectorization
### CountVectorizer / TfidfVectorizer

https://www.kaggle.com/ankkur13/prediction-based-on-bayes-algo-nlp-wordcloud
https://www.kaggle.com/adamschroeder/countvectorizer-tfidfvectorizer-predict-comments

In [204]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
count_vec = CountVectorizer()
feature_train_counts = count_vec.fit(raw['Text'])
bag_of_words = feature_train_counts.transform(raw['Text'])

In [205]:
#print("Every feature:\n{}".format(count_vec.get_feature_names()))
print("\nEvery 3rd feature:\n{}".format(count_vec.get_feature_names()[::3]))
#print("Vocabulary size: {}".format(len(feature_train_counts.vocabulary_)))
#print("Vocabulary content:\n {}".format(feature_train_counts.vocabulary_))


Every 3rd feature:
['00', '0000a', '002', '002mm', '0031de', '0048', '008', '0100', '0105', '0130', '01500', '0200da', '0210', '025007', '02de', '0300', '032w', '0376', '040', '043', '05', '051ios', '05de', '0600', '061216', '068', '07', '07001830', '0700da', '0730', '075', '0798', '0800', '080malik', '082016', '087', '090', '090418', '0906', '0913', '093', '0945', '0950', '096', '0a', '0kuyucu', '0n', '0x0', '0x0telefonal', '0x1in', '100', '100000', '10001100', '10001500tl', '1000da', '1000tllik', '100120', '1002', '1005', '100den', '100dolar', '100gbit', '100le', '100lüyorum', '100olarak', '101', '1010alan', '1010çift', '1015', '1017', '1020', '103', '1033', '1045', '1048', '1052g', '1070', '1080', '1080p60fps', '1084', '108gb', '109', '1099', '109ram', '10amp8217un', '10cm', '10daürün', '10dk', '10dur', '10gb', '10gün', '10kamera', '10larca', '10mbps', '10ncu', '10npro', '10numara5yıldız', '10parmak', '10puan', '10sağlam', '10tlye', '10unda', '10x', '10yıla', '10ön', '110', '1100de

In [213]:
bag_of_words

<44317x101311 sparse matrix of type '<class 'numpy.int64'>'
	with 1464042 stored elements in Compressed Sparse Row format>

In [32]:
from sklearn.model_selection import train_test_split
X = bag_of_words 
y = raw['Rating'].fillna('')
feature_train, feature_test, label_train, label_test = train_test_split(X, y, test_size=0.33, shuffle=True)

[<1x64 sparse matrix of type '<class 'numpy.int64'>'
 	with 30 stored elements in Compressed Sparse Row format>,
 <1x64 sparse matrix of type '<class 'numpy.int64'>'
 	with 40 stored elements in Compressed Sparse Row format>]

## TFIDFVectorizer

In [231]:
trans_tfidf

<44317x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 2361369 stored elements in Compressed Sparse Row format>

In [230]:
tfidf = TfidfVectorizer(max_features=100000,token_pattern='\w+',ngram_range=(1, 2))
tfidf_fitted = tfidf.fit(raw['Text'])
trans_tfidf = tfidf_fitted.transform(raw['Text'])
#trans_tfidf = tfidf_fitted.transform(feature_test)

In [ ]:
from sklearn.model_selection import train_test_split
X = trans_tfidf
y = raw['Rating'].fillna('')
feature_train, feature_test, label_train, label_test = train_test_split(X, y, test_size=0.33, shuffle=True)

In [ ]:
raw['Rating'].value_counts()

In [184]:
raw.to_csv('postdata_1.csv')

In [ ]:
#tfidf_fitted.vocabulary_
# 2 harfli keyleri sil, sayılar hariç, 'gb' hariç, ? hariç

In [ ]:
trans_tfidf.data.reshape(-1, 1)[0]

In [ ]:
trans_tfidf.shape

# Model Building

### MultinomialNB

In [33]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(feature_train, label_train)
preds_mnb = clf.predict(feature_test)

from sklearn.metrics import confusion_matrix, classification_report

print('Valid RMSLE: {:.4f}'.format(np.sqrt(mean_squared_log_error(label_test, preds_mnb))))
print(confusion_matrix(label_test, preds_mnb))
print('\n')
print(classification_report(label_test, preds_mnb))

Valid RMSLE: 0.2962
[[   0    0   22   63  224]
 [   0    0   13   80  142]
 [   0    0   53  247 1161]
 [   0    0   20  634 1994]
 [   0    0   44  237 9691]]


             precision    recall  f1-score   support

         20       0.00      0.00      0.00       309
         40       0.00      0.00      0.00       235
         60       0.35      0.04      0.07      1461
         80       0.50      0.24      0.32      2648
        100       0.73      0.97      0.84      9972

avg / total       0.63      0.71      0.64     14625



c:\users\dogus\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
tree = RandomForestClassifier()
trees = tree.fit(feature_train,label_train)  #ValueError: Number of labels=14625 does not match number of samples=29692
preds_forest = trees.predict(feature_test)

from sklearn.metrics import confusion_matrix, classification_report
print('Valid RMSLE: {:.4f}'.format(np.sqrt(mean_squared_log_error(label_test, preds_forest))))
print(confusion_matrix(label_test, preds_forest))
print('\n')
print(classification_report(label_test, preds_forest))

# Pipeline Method

In [189]:
#http://scikit-learn.org/stable/modules/linear_model.html
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
df = pd.read_csv('postdata_1.csv')

In [199]:
#pd.Series([1, 2, 3]).values

X = df['Text']
y = df['Rating']
feature_train, feature_test, label_train, label_test = train_test_split(X, y, test_size=0.33, shuffle=True)

pipeline = Pipeline([
    ('vect', CountVectorizer()),                   # strings to token integer counts
    #('tfidf', TfidfVectorizer()),                 # integer counts to weighted TF-IDF scores
    #('clf', MultinomialNB()),                     # train on TF-IDF vectors w/ Naive Bayes classifier
    #('clf', SGDClassifier(loss="log")), RMSLE: 0.2493, F1: 0.73, log_loss=0.75
    #('clf', SVC(C=10,gamma=0.01)),                #F1: 0.74 
    ('clf',Lasso(alpha=0.1)), # RMSLE: 0.2407, continuous predictions
    #('clf',BayesianRidge()),
    #('clf',LogisticRegression()),  # RMSLE: 0.2412, F1: 0.75
    #('clf',SGDRegressor()),  # RMSLE: 0.2321
    #('clf',Perceptron()),   # RMSLE: 0.2523, F1: 0.73
    #('clf',PassiveAggressiveClassifier()),  # RMSLE: 0.2412, F1:0.73
    #('clf',PassiveAggressiveRegressor()),
    #('clf',TheilSenRegressor()),
    #('clf',HuberRegressor()),
    #('clf',RANSACRegressor()),
    #('clf',Ridge()),
    #('clf',ElasticNet()),  # RMSLE: 0.2713
])

pipeline.fit(feature_train, label_train)
preds = pipeline.predict(feature_test)

"""from sklearn.metrics import confusion_matrix, classification_report
print('Valid RMSLE: {:.4f}'.format(np.sqrt(mean_squared_log_error(label_test, preds))))
print(confusion_matrix(label_test, preds))
print('\n')
print(classification_report(label_test, preds))"""

preds_proba = pipeline.predict_proba(feature_test)
print("F1_score : {:.2%} ".format(f1_score(label_test, preds, average='micro')))  # 'samples', 'weighted', 'macro', 'micro', 'binary'
print("Log_loss : {:.4f} ".format(log_loss(label_test, preds_proba)))

Valid RMSLE: 0.2564


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

# Keras API
API:application program interface

In [234]:
import keras as ks
from keras.models import Sequential
from keras.layers import Dense


model = Sequential()

model_in = ks.Input(shape=(trans_tfidf.shape[1],), dtype='float32', sparse=True)

model.add(Dense(units=10, activation='relu',input_dim=100000))
model.add(Dense(units=3, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy']
)

In [235]:
model.fit(feature_train, label_train, epochs=5, batch_size=32)

ValueError: Error when checking input: expected dense_9_input to have shape (100000,) but got array with shape (1,)

# GridSearchCV

In [130]:
from sklearn.model_selection import GridSearchCV
import logging
from sklearn.svm import SVC

pipeline = Pipeline([
    ('vect', CountVectorizer()),  # strings to token integer counts
    #('tfidf', TfidfVectorizer()),  # integer counts to weighted TF-IDF scores
    #('clf', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
    #('clf', SGDClassifier()),
    ('clf', SVC()),
])

parameters_sgd = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}


#https://medium.com/@aneesha/svm-parameter-tuning-in-scikit-learn-using-gridsearchcv-2413c02125a0
parameters_svc = {
        'clf__C':(0.001, 0.01, 0.1, 1, 10),
    'clf__gamma':(0.001, 0.01, 0.1, 1),
    }

#parameters_NB = {



In [129]:
parameters = parameters_svc
if __name__=='__main__':
    # multiprocessing requires the fork to happen in a __main__ protected
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=3, verbose=1)
    print("Performing grid search...")
    print('pipeline:',[name for name, _ in pipeline.steps])
    print('parameters:')
    print(parameters)
    t0=time.time()
    grid_search.fit(feature_train, label_train)
    print("done in %0.3fs" % grid_search.best_score_)
    print('Best parameters set:')
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
else:print('WHY')
    
# What is:
# \t
# %r
# sorted()


Performing grid search...
pipeline: ['vect', 'clf']
parameters:
{'clf__C': (0.001, 0.01, 0.1, 1, 10), 'clf__gamma': (0.001, 0.01, 0.1, 1)}
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 127.5min
[Parallel(n_jobs=3)]: Done  60 out of  60 | elapsed: 219.0min finished


done in 0.761s
Best parameters set:
	clf__C: 10
	clf__gamma: 0.01


In [122]:
parameters.keys()

dict_keys(['vect__max_df', 'vect__ngram_range', 'clf__alpha', 'clf__penalty'])

In [137]:
from sklearn.svm import SVC
def svc_param_tuning(X, y,pipeline):
    param_grid = dict(clf__C = (10,20,30,40),
                      clf__gamma = (0.01,0.01,),)

    grid_search = GridSearchCV(pipeline, param_grid, n_jobs=3, verbose=1)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_
print(svc_param_tuning(feature_train,label_train,pipeline=pipeline))
#{'clf__C': 10, 'clf__gamma': 0.01}
#def sgd_param_tuning(X, y):

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=3)]: Done  24 out of  24 | elapsed: 95.6min finished


{'clf__C': 10, 'clf__gamma': 0.01}


In [109]:
SVC().get_params()

{'C': 1.0,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'auto',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

# Methods & Try Outs

https://stackoverflow.com/questions/39001956/sklearn-pipeline-how-to-apply-different-transformations-on-different-columns
* You have to define your functions with def since annoyingly you can't use lambda or partial in FunctionTransformer if you want to pickle your model
* You need to initialize FunctionTransformer with validate=False

In [ ]:
"""#https://stackoverflow.com/questions/39007083/not-sure-how-to-use-sklearn-with-feature-vectors-which-contain-both-text-and-num

from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer,LabelEncoder,MinMaxScaler,StandardScaler
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

def get_text_cols(df):
    return df[['Location', 'Comment', 'CommentTitle', 'Brand', 'Gender','Model']]

def get_num_cols(df):
    return df[['p_helpful', 'n_helpful']]
def get_age_col(df):
    return df['Age']
vec = make_union(*[
    make_pipeline(FunctionTransformer(get_text_cols, validate=False), LabelEncoder()),
    make_pipeline(FunctionTransformer(get_num_cols, validate=False), MinMaxScaler()),
    make_pipeline(FunctionTransformer(get_age_col, validate=False), MinMaxScaler())
])

vector = vec.fit_transform(X_train)"""

In [214]:
def audio_dogus(path_to_file=r"C:\Users\dogus\Dropbox\DgsPy_DBOX\clock-chimes.wav"):
    import simpleaudio as sa
    #play_obj = sa.play_buffer("C:/Users/dogusMusic/clock-chimes.wav", 2, 2, 44100)
    wave_obj = sa.WaveObject.from_wave_file(path_to_file)
    play_obj = wave_obj.play()

from multiprocessing.pool import ThreadPool
with ThreadPool(processes=4) as pool:

In [ ]:
def alarm_dogus():
    import 

# Graveyard

In [ ]:
"""# Clean out all the punctuations
import string
def punc_clean(text):
    stext = [word for word in text if word not in string.punctuation]
    ftext = ''.join(stext)
    return ftext

# Clean out all the stopwords
from nltk.corpus import stopwords
def stop_tr_clean(text):
    stext = [word for word in text.split() if word.lower() not in stopwords.words('turkish')]
    ftext = ' '.join(stext)
    return ftext
t0 = time.time()
raw['Comment'] = raw['Comment'].apply(punc_clean)
#raw['Comment'] = raw['Comment'].apply(stop_tr_clean) # 9:53
print("done in %0.3fs." % (time.time() - t0))"""

In [ ]:
"""for i in range(1,len(feature_train)):
    try:
        print(feature_train.iloc[i].lower())
    except:
        print(i, "didnt work")"""

In [ ]:
#[word for word in textt.split() if word.lower() not in stopwords.words('turkish')]